# Llama Stack - Agents Evaluation

In [37]:
import os
# from llama_stack.distribution.library_client import LlamaStackAsLibraryClient
from llama_stack_client import LlamaStackClient

# client = LlamaStackAsLibraryClient(
#     "together", provider_data = {"tavily_search_api_key": os.environ['TAVILY_SEARCH_API_KEY']})

client = LlamaStackClient(base_url="http://localhost:8321")
# _ = client.initialize()

#### Building a Search Agent

In [38]:
from llama_stack_client.types.agent_create_params import AgentConfig
from llama_stack_client.lib.agents.agent import Agent

model_id = "meta-llama/Llama-3.3-70B-Instruct"

agent_config_with_websearch = AgentConfig(
    model=model_id,
    instructions="You are a helpful assistant that can answer questions by searching the web.",
    sampling_params={
        "strategy": {"type": "top_p", "temperature": 1.0, "top_p": 0.9},
    },
    toolgroups=["builtin::websearch"],
    tool_config={
        "tool_choice": "auto",
        "tool_prompt_format": "python_list",
    },
    input_shields=[],
    output_shields=[],
    enable_session_persistence=False,
)

In [52]:
import uuid
from datasets import load_dataset
from rich.pretty import pprint

# 0. Create an agent
agent = Agent(client, agent_config_with_websearch)

# 2. Send it a couple of questions in the sesison. Here, we get the first 10 questions from the Simple QA dataset.
ds = load_dataset("llamastack/evals", "evals__simpleqa")
questions = []
answers = []
for idx, x in enumerate(ds["train"]):
    if idx >= 3:
        break
    questions.append(x["input_query"])
    answers.append(x["expected_answer"])

for query in questions:
    # create a new session for each question
    session_id = agent.create_session(f"{uuid.uuid4().hex}")
    response = agent.create_turn(
        [
            {
                "role": "user",
                "content": query,
            }
        ], 
        session_id=session_id,
        stream=False
    )
    pprint(response)

Turn(
│   input_messages=[
│   │   UserMessage(content='Who received the IEEE Frank Rosenblatt Award in 2010?', role='user', context=None)
│   ],
│   output_message=CompletionMessage(
│   │   content='The recipient of the IEEE Frank Rosenblatt Award in 2010 was Dr. Ronald Fagin.',
│   │   role='assistant',
│   │   stop_reason='end_of_turn',
│   │   tool_calls=[]
│   ),
│   session_id='3a15bc56-f736-4126-8137-a5cb87406225',
│   started_at=datetime.datetime(2025, 2, 26, 12, 48, 15, 774381, tzinfo=TzInfo(-08:00)),
│   steps=[
│   │   InferenceStep(
│   │   │   api_model_response=CompletionMessage(
│   │   │   │   content='The recipient of the IEEE Frank Rosenblatt Award in 2010 was Dr. Ronald Fagin.',
│   │   │   │   role='assistant',
│   │   │   │   stop_reason='end_of_turn',
│   │   │   │   tool_calls=[]
│   │   │   ),
│   │   │   step_id='6c660138-574e-49a9-9f8c-552ccce874fc',
│   │   │   step_type='inference',
│   │   │   turn_id='7f07185b-66a0-4701-b965-5ff74793c2c4',
│   │   │   completed_at=datetime.datetime(2025, 2, 26, 12, 48, 16, 689502, tzinfo=TzInfo(-08:00)),
│   │   │   started_at=datetime.datetime(2025, 2, 26, 12, 48, 15, 786046, tzinfo=TzInfo(-08:00))
│   │   )
│   ],
│   turn_id='7f07185b-66a0-4701-b965-5ff74793c2c4',
│   completed_at=datetime.datetime(2025, 2, 26, 12, 48, 16, 702110, tzinfo=TzInfo(-08:00)),
│   output_attachments=[]
)

Turn(
│   input_messages=[
│   │   UserMessage(
│   │   │   content="Who was awarded the Oceanography Society's Jerlov Award in 2018?",
│   │   │   role='user',
│   │   │   context=None
│   │   )
│   ],
│   output_message=CompletionMessage(
│   │   content="The Oceanography Society's Jerlov Award for 2018 was awarded to Dr. Emmanuel Boss.",
│   │   role='assistant',
│   │   stop_reason='end_of_turn',
│   │   tool_calls=[]
│   ),
│   session_id='d0dfe88d-f75c-448c-892a-067aff1c1b72',
│   started_at=datetime.datetime(2025, 2, 26, 12, 48, 16, 740702, tzinfo=TzInfo(-08:00)),
│   steps=[
│   │   InferenceStep(
│   │   │   api_model_response=CompletionMessage(
│   │   │   │   content="The Oceanography Society's Jerlov Award for 2018 was awarded to Dr. Emmanuel Boss.",
│   │   │   │   role='assistant',
│   │   │   │   stop_reason='end_of_turn',
│   │   │   │   tool_calls=[]
│   │   │   ),
│   │   │   step_id='c9c7ede1-94af-4f28-9068-8817dd675392',
│   │   │   step_type='inference',
│   │   │   turn_id='f701695e-389c-4078-9a4f-228479cf4d6a',
│   │   │   completed_at=datetime.datetime(2025, 2, 26, 12, 48, 17, 505687, tzinfo=TzInfo(-08:00)),
│   │   │   started_at=datetime.datetime(2025, 2, 26, 12, 48, 16, 752172, tzinfo=TzInfo(-08:00))
│   │   )
│   ],
│   turn_id='f701695e-389c-4078-9a4f-228479cf4d6a',
│   completed_at=datetime.datetime(2025, 2, 26, 12, 48, 17, 516772, tzinfo=TzInfo(-08:00)),
│   output_attachments=[]
)

Turn(
│   input_messages=[
│   │   UserMessage(
│   │   │   content="What's the name of the women's liberal arts college in Cambridge, Massachusetts?",
│   │   │   role='user',
│   │   │   context=None
│   │   )
│   ],
│   output_message=CompletionMessage(
│   │   content="The women's liberal arts college in Cambridge, Massachusetts is Radcliffe College. However, it's important to note that Radcliffe College merged with Harvard University in 1999 and is now known as the Radcliffe Institute for Advanced Study at Harvard University. The institute still offers programs and resources for women, but it is no longer an independent college.",
│   │   role='assistant',
│   │   stop_reason='end_of_turn',
│   │   tool_calls=[]
│   ),
│   session_id='2424c5ca-afb2-4cc4-a5f5-f7ce0ec473d5',
│   started_at=datetime.datetime(2025, 2, 26, 12, 48, 17, 560298, tzinfo=TzInfo(-08:00)),
│   steps=[
│   │   InferenceStep(
│   │   │   api_model_response=CompletionMessage(
│   │   │   │   content="The women's liberal arts college in Cambridge, Massachusetts is Radcliffe College. However, it's important to note that Radcliffe College merged with Harvard University in 1999 and is now known as the Radcliffe Institute for Advanced Study at Harvard University. The institute still offers programs and resources for women, but it is no longer an independent college.",
│   │   │   │   role='assistant',
│   │   │   │   stop_reason='end_of_turn',
│   │   │   │   tool_calls=[]
│   │   │   ),
│   │   │   step_id='2cbf44bc-d836-4449-99f6-a5fe72c2d8a0',
│   │   │   step_type='inference',
│   │   │   turn_id='0b4040e7-7aa6-4f86-a072-31333ffa29be',
│   │   │   completed_at=datetime.datetime(2025, 2, 26, 12, 48, 19, 269112, tzinfo=TzInfo(-08:00)),
│   │   │   started_at=datetime.datetime(2025, 2, 26, 12, 48, 17, 571880, tzinfo=TzInfo(-08:00))
│   │   )
│   ],
│   turn_id='0b4040e7-7aa6-4f86-a072-31333ffa29be',
│   completed_at=datetime.datetime(2025, 2, 26, 12, 48, 19, 282118, tzinfo=TzInfo(-08:00)),
│   output_attachments=[]
)

### Examine agent logs

In [53]:
# get the agent session ids
session_ids = agent.sessions
agent_id = agent.agent_id

# print(session_ids)
response = client.agents.session.retrieve(
    session_id="d1cb76fb-cd59-4dd9-afe5-0ce7ede630a9",
    agent_id=agent_id,
)
print(response)
# client.agents.turn.retrieve(
#     session_id="89459cef-7494-4a37-8278-59343b6174a3",
#     agent_id=agent_id,
#     turn_id="f1e09cd5-a2af-483b-9dfa-cc374528bcd7",
# )
# session2turns = {}
# for session_id in session_ids:
#     session2turns[session_id] = client.agents.session.retrieve(
#         session_id=session_id,
#         agent_id=agent_id,
#     )

# pprint(session2turns)

InternalServerError: Error code: 500 - {'detail': 'Internal server error: An unexpected error occurred.'}